<a href="https://colab.research.google.com/github/NehaNeha417/loan_prediction/blob/master/Flickr_8k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import string
import os
import numpy as np
from PIL import Image
from pickle import dump, load

from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers.merge import add
from keras.models import Model, load_model
from keras.layers import Dense, Dropout, Input, LSTM, Embedding
from tqdm import tqdm_notebook as tqdm
tqdm().pandas()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  app.launch_new_instance()


In [ ]:
def load_doc(filename):
  file = open(filename,'r')
  text = file.read()
  file.close()
  return text

def img_all_captions(filename):
  doc = load_doc(filename)
  captions = doc.split('\n')
  dictionary = {}
  for caption in captions[:-1]:
    img, cap = caption.split('\t')
    if img[:-2] not in dictionary:
      dictionary[img[:-2]] = [cap]
    else:
      dictionary[img[:-2]].append(cap)
  return dictionary

def cleaning_text(dictionary):
  #table = str.maketrans('','', string.punctuation)
  for img, cap in dictionary.items():
    for i, cap in enumerate(cap):
      str = cap[i]
      table = str.maketrans('','', string.punctuation)
      cap.replace('-',' ')
      words = cap.split()
      new = []
      for word in words:
        word.lower()
        word.translate(table)
        if len(word)>1 and word.isalpha():
          new.append(word)
      cap = ' '.join(new)
      dictionary[img][i] = cap
  return dictionary


def text_vocabulary(dictionary):
  vocab = set()
  for img in dictionary.keys():
    for lines in dictionary[img]:
      tokens = lines.split()
      [vocab.update(tokens)]
  return vocab


def save_description(dictionary,new_file):
  lines = list()
  for img, cap in dictionary.items():
    for line in cap:
      lines.append(img + '\t' + line)
  data = ('\n').join(lines)
  file = open('new_file','w')
  file.write(data)
  #print(data)
  file.close()
  
data_img = '/content/drive/My Drive/Flicker_8K/Flicker8k_Dataset (1)'
data_txt = '/content/drive/My Drive/Flicker_8K/Flickr8k_text'

token_file = data_txt + '/' + 'Flickr8k.token.txt'
dictionary = img_all_captions(token_file)
print('length of dictionary', len(dictionary))
clean_text = cleaning_text(dictionary)
vocabulary =text_vocabulary(clean_text)
print('length of vocabulary', len(vocabulary))
save_description(clean_text, 'new_file')
print(dictionary)
print(vocabulary)

length of dictionary 8092
length of vocabulary 9052
{'1000268201_693b08cb0e.jpg': ['child in pink dress is climbing up set of stairs in an entry way', 'girl going into wooden building', 'little girl climbing into wooden playhouse', 'little girl climbing the stairs to her playhouse', 'little girl in pink dress going into wooden cabin'], '1001773457_577c3a7d70.jpg': ['black dog and spotted dog are fighting', 'black dog and dog playing with each other on the road', 'black dog and white dog with brown spots are staring at each other in the street', 'Two dogs of different breeds looking at each other on the road', 'Two dogs on pavement moving toward each other'], '1002674143_1b742ab4b8.jpg': ['little girl covered in paint sits in front of painted rainbow with her hands in bowl', 'little girl is sitting in front of large painted rainbow', 'small girl in the grass plays with fingerpaints in front of white canvas with rainbow on it', 'There is girl with pigtails sitting in front of rainbow pai

In [ ]:
file = open('new_file','r')



In [ ]:
def extract_features(img_file):
  model = Xception(include_top=False, pooling='avg')
  features = {}
  for img in tqdm(os.listdir(img_file)):
    new_img_file = img_file + '/' + img
    image = Image.open(new_img_file,'r')
    image = image.resize((299,299))
    image = np.expand_dims(image, axis=0)
    image = image/127.5
    image = image - 1.0
    
    feature = model.predict(image)
    features[img] = feature
  return features

feature_vector = extract_features(data_img)
dump(feature_vector, open('features.p','wb'))


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [ ]:
pickle_off = open("features.p", 'rb')
new_features = load(pickle_off)
print(new_features)

{'1016887272_03199f49c4.jpg': array([[5.56079485e-02, 2.05999151e-01, 1.24200374e-01, ...,
        6.04230985e-02, 4.82259493e-05, 3.11633330e-02]], dtype=float32), '1055753357_4fa3d8d693.jpg': array([[0.        , 0.175389  , 0.03379288, ..., 0.        , 0.13600987,
        0.07780214]], dtype=float32), '1032460886_4a598ed535.jpg': array([[2.5502997e-02, 1.3933477e+00, 4.2630425e-01, ..., 8.8139087e-02,
        0.0000000e+00, 1.2905434e-03]], dtype=float32), '1117972841_2b9261f95f (1).jpg': array([[0.52321035, 0.37000382, 0.07632015, ..., 0.18410076, 0.11709175,
        0.        ]], dtype=float32), '1082252566_8c79beef93.jpg': array([[0.3599714 , 0.02476221, 0.09008387, ..., 0.33866078, 0.06513867,
        0.0104998 ]], dtype=float32), '1115679311_245eff2f4b (1).jpg': array([[0.        , 0.37079266, 0.04244021, ..., 0.01810757, 0.45136604,
        0.        ]], dtype=float32), '1084040636_97d9633581.jpg': array([[0.        , 0.11441895, 0.00711011, ..., 0.0628791 , 0.47133482,
       

In [ ]:
def load_photos(train_text_file):
  file = load_doc(train_text_file)
  photos = file.split('\n')[:-1]
  return photos


def load_clean_description(description_file, photos):
  file = load_doc(description_file)
  new_description = {}
  for line in file.split('\n'):
    words = line.split()
    if len(words) < 1:
      continue
    img, img_cap = words[0], words[1:]
    if img in photos:
      if img not in new_description:
        new_description[img] = []
      desc = '<START>' + ' '.join(img_cap) + '<END>'
      new_description[img].append(desc)
  return new_description


def load_features(photos):
  all_features = load(open('features.p','rb'))
  for photo in photos:
    photo_feature = {photo:all_features[photo]}
  return photo_feature

train_text_file = data_txt + '/' + 'Flickr_8k.trainImages.txt'
train_images = load_photos(train_text_file)

train_description = load_clean_description('new_file',train_images)
train_features = load_features(train_images)


KeyError: ignored

In [ ]:
def dict_to_list(dictionary):
  train_list = []
  for key in dictionary.keys():
    for line in dictionary[key]:
      train_list.append(line)

def create_tokenizer(train_list):
  new_list = dict_to_list(train_list)
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(new_list)
  return tokenizer

tokenizer = create_tokenizer(train_description)
dump(tokenizer, load(open('tokenizer.p','wb')))
vocab_size = len(tokenizer.word_index) + 1

print(vocab_size)


def max_length_description(description):
  maximum = []
  for lines in train_description.values():
    for line in lines:
      num = len(line.split())
      maximum.append(num)
  max_length = max(maximum)

print(max_length_description(train_description))


In [ ]:
def data_generator(train_description, train_features, tokenizer, max_length):
  while 1:
    for key,value_list in train_description.items():
    feature = train_features[key][0]
    input_image, input_sequence, output_word = create_sequences(tokenizer, max_length, value_list, train_features)
    yield [[input_image, input_sequence],output_word]

def create_sequences(tokenizer, max_length, value_list, train_features):
  X1, X2, y = list(), list(), list()
  for desc in value_list:
    seq = tokenizer.texts_to_sequences([desc])[0]
    for i in range(1, len(seq)):
      in_seq, out_seq = seq[:i], seq[i]
      in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
      out_seq = to_categorical([out_seq], num_classses=vocab_size)[0]

      X1.append(feature)
      X2.append(in_seq)
      y.append(out_seq)
  return np.array(X1), np.array(X2), np.array(y)

[a,b], c = next(data_generator(train_description, train_features, tokenizer, max_length))
print(a.shape)
print(b.shape)
print(c.shape)


In [ ]:
from keras.utils import plot_model
def define_model(vocab_size, max_length):
  inputs1 = Input(shape(2048, ))
  fe1 = Dropout(0.5)(inputs1)
  fe2 = Dense(256,activation='relu')(fe1)

  inputs2 = Input(shape(max_length, ))
  se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
  se2 = Dropout(0.5)(se1)
  se3 = LSTM(256)(se2)

  decoder1 = add([fe2,se3])
  decoder2 = Dense(256, activation='relu')(decoder1)
  outputs = Dense(vocab_size, activation='relu')(decoder2)

  model = Model(input=[inputs1,inputs2], output=outputs)
  model.compile(loss='cross', optimizer='adam')
  
  print(model.summary())
  plot_model(model, to_file='model.png', show_shapes=true)

  return model

In [ ]:
print('Dataset:', train_images)
print('Descriptions Train:', train_descriptions)
print('Features Train:', len(train_features))
print('Vocabulary_size:', vocab_size)
print('Descrption Length:', max_length)

model = define_model(vocab_size, max_length)
epochs = 10
steps = len(train_description)
os.mkdir('models')
for i in range(epochs):
  generator = data_generator(train_description, train_features, tokenizer, max_length)
  model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
  model.save('models/model_' + str(i) + '.h5')


In [ ]:
import argpars
import matplotlib.pyplot as plt

ap = argparse.ArgumentParser()
ap.add_argument('-i','--image', required=True, help='Image Path')
args = vars(ap.parse_args())
img_path = args['image']

def extract_features(filename, model):
  try:
    image = Image.open(filename)
  except:
    print('There is some error')
  image = image.resize((299,299))
  image = np.array(image)
  if image.shape[2] == 4:
    image = image[..., :3]
    image = np.expand_dims(image, axis=0)
    image = image/127.5
    image = image - 1.0
    feature = model.predict(image)
    return feature

def word_for_id(integer, tokenizer):
  for word,index in tokenizer.word_index.items():
    if index == integer:
      return word
    return None


def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'start'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        pred = model.predict([photo,sequence], verbose=0)
        pred = np.argmax(pred)
        word = word_for_id(pred, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'end':
            break
    return in_text


max_length = 32
tokenizer = load(open("tokenizer.p","rb"))
model = load_model('models/model_9.h5')
xception_model = Xception(include_top=False, pooling="avg")
photo = extract_features(img_path, xception_model)
img = Image.open(img_path)
description = generate_desc(model, tokenizer, photo, max_length)
print("\n\n")
print(description)
plt.imshow(image)

